<a href="https://colab.research.google.com/github/DevSingh28/Spam-or-Not-Spam-Mail-detector/blob/main/spam_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
df = pd.read_csv('spam_ham_dataset.csv')

In [3]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
df['text'][3]

'Subject: photoshop , windows , office . cheap . main trending\r\nabasements darer prudently fortuitous undergone\r\nlighthearted charm orinoco taster\r\nrailroad affluent pornographic cuvier\r\nirvin parkhouse blameworthy chlorophyll\r\nrobed diagrammatic fogarty clears bayda\r\ninconveniencing managing represented smartness hashish\r\nacademies shareholders unload badness\r\ndanielson pure caffein\r\nspaniard chargeable levin\r\n'

In [5]:
df.shape

(5171, 4)

In [6]:
df.isna().values.any()

False

In [7]:
df.duplicated().values.any()

False

In [8]:
X = df['text']
y = df['label_num']

In [9]:
type(X), type(y)

(pandas.core.series.Series, pandas.core.series.Series)

In [10]:
X = df['text'].values
y = df['label_num'].values

In [11]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=200)

In [14]:
X_train.shape, X_test.shape, y_train.shape

((3619,), (1552,), (3619,))

In [15]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vect = TextVectorization(max_tokens=None, standardize='lower_and_strip_punctuation', split='whitespace', ngrams=None, output_mode='int', output_sequence_length=None, pad_to_max_tokens=False)

In [17]:
round(sum([len(i.split()) for i in X])/len(X))

228

In [18]:
text_vect = TextVectorization(max_tokens=10000, output_mode='int', output_sequence_length=228, pad_to_max_tokens=True)

In [19]:
text_vect.adapt(X)

In [20]:
text_vect(X[0])

<tf.Tensor: shape=(228,), dtype=int64, numpy=
array([  10,   16,  889,   44,    1,   15,   14,    8,  666,   77,    3,
          2,  430,   17, 1831,    9,   12,  407,   68,   39,   45, 1799,
        159,  449,  673,   37,   60,   32, 8222, 2357,   29,  184,  126,
       1822,  657,    3, 1134,  184,  392,    9,   55, 3007,   22,   36,
        534,   15,  153,   14,  498, 1431,    6, 4031, 1883,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [21]:
emb = layers.Embedding(input_dim=10000, output_dim=128, input_length=228)

In [22]:
emb(text_vect(X[0]))

<tf.Tensor: shape=(228, 128), dtype=float32, numpy=
array([[ 0.00383637, -0.0322603 , -0.02270957, ...,  0.02696938,
        -0.04977689,  0.02322812],
       [ 0.04564101,  0.04696418, -0.04667691, ..., -0.04798719,
        -0.00072319,  0.01123383],
       [-0.02064596,  0.01503899,  0.04818107, ...,  0.0033751 ,
         0.0324722 ,  0.0085762 ],
       ...,
       [ 0.02189876, -0.03210099,  0.04548258, ..., -0.01971909,
        -0.03197384,  0.01515346],
       [ 0.02189876, -0.03210099,  0.04548258, ..., -0.01971909,
        -0.03197384,  0.01515346],
       [ 0.02189876, -0.03210099,  0.04548258, ..., -0.01971909,
        -0.03197384,  0.01515346]], dtype=float32)>

In [23]:
#Lets first test this with inbuild model first

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [24]:
pipe = Pipeline(
    [('tfid', TfidfVectorizer()),
     ('clasi', MultinomialNB())]
)

In [25]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('tfid', TfidfVectorizer()), ('clasi', MultinomialNB())])

In [26]:
pred = pipe.predict(X_test)

In [27]:
new_pred = np.round(pred).astype(int)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, new_pred))
print(confusion_matrix(y_test, new_pred))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1101
           1       1.00      0.56      0.72       451

    accuracy                           0.87      1552
   macro avg       0.92      0.78      0.82      1552
weighted avg       0.89      0.87      0.86      1552

[[1101    0]
 [ 197  254]]


In [48]:
def classify_email(user_input):

    emb(text_vect(np.array([user_input])))
    predicted_label = pipe.predict([user_input])
    if predicted_label == 1:
        classification = "SPAM"
        print("The email is classified as:", classification)
    else:
        classification = "NOT SPAM"
        print("The email is classified as:", classification)

user_input = input("Enter the email message to classify: ")
classify_email(user_input)


Enter the email message to classify: helolo
The email is classified as: NOT SPAM


In [ ]:
# Creating own LSTM rnn model
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vect(inputs)
x = emb(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
114/114 [==============================] - 100s 743ms/step - loss: 0.4958 - val_loss: 0.3539
Epoch 2/15
114/114 [==============================] - 63s 554ms/step - loss: 0.3447 - val_loss: 0.3558
Epoch 3/15
114/114 [==============================] - 63s 554ms/step - loss: 0.3447 - val_loss: 0.3559
Epoch 4/15
114/114 [==============================] - 57s 500ms/step - loss: 0.3433 - val_loss: 0.3514
Epoch 5/15
114/114 [==============================] - 62s 543ms/step - loss: 0.3433 - val_loss: 0.3493
Epoch 6/15
114/114 [==============================] - 57s 501ms/step - loss: 0.3379 - val_loss: 0.3478
Epoch 7/15
114/114 [==============================] - 61s 537ms/step - loss: 0.3384 - val_loss: 0.3490
Epoch 8/15
114/114 [==============================] - 59s 520ms/step - loss: 0.3355 - val_loss: 0.3464
Epoch 9/15
114/114 [==============================] - 57s 500ms/step - loss: 0.3346 - val_loss: 0.3416
Epoch 10/15
114/114 [==============================] - 60s 527ms/step - 

In [ ]:
pred2 = model.predict(X_test)

49/49 [==============================] - 13s 199ms/step


In [ ]:
new_pred2 = np.round(pred).astype(int)

In [ ]:
print(classification_report(y_test, new_pred2))
print(confusion_matrix(y_test, new_pred2))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1101
           1       1.00      0.56      0.72       451

    accuracy                           0.87      1552
   macro avg       0.92      0.78      0.82      1552
weighted avg       0.89      0.87      0.86      1552

[[1101    0]
 [ 197  254]]


In [29]:
# Lets improve it

In [39]:
from tensorflow.keras.callbacks import EarlyStopping
early = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)

In [ ]:
text_vect.adapt(X_train)
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vect(inputs)
x = emb(x)
x = layers.LSTM(128, return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.LSTM(128)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), callbacks=[early])

Epoch 1/15
114/114 [==============================] - 122s 1s/step - loss: 0.3871 - accuracy: 0.8563 - val_loss: 0.2611 - val_accuracy: 0.9317
Epoch 2/15
114/114 [==============================] - 117s 1s/step - loss: 0.3112 - accuracy: 0.9077 - val_loss: 0.3320 - val_accuracy: 0.8950
Epoch 2: early stopping


In [ ]:
pred3 = model.predict(X_test)

49/49 [==============================] - 13s 272ms/step


In [ ]:
new_pred3 = np.round(pred3).astype(int)

In [ ]:
print(classification_report(y_test, new_pred3))
print(confusion_matrix(y_test, new_pred3))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      1101
           1       0.95      0.67      0.79       451

    accuracy                           0.89      1552
   macro avg       0.92      0.83      0.86      1552
weighted avg       0.90      0.89      0.89      1552

[[1085   16]
 [ 147  304]]


In [54]:
#Lets try one more inbuild model

# Pretrained Feature Extractor Universal Sentence Encoder (USE) model from TensorFlow Hub

In [32]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")



In [36]:
checking = embed(['Hello My name is Dev Singh'])
checking[0][:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.00833142, -0.04881498,  0.0426286 ,  0.05782176,  0.04933576,
       -0.0134643 , -0.01335356, -0.03883906,  0.05751929, -0.06230747],
      dtype=float32)>

In [55]:
emb_final = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape=[], dtype=tf.string, trainable=False
)

In [41]:
final_model = tf.keras.Sequential([
    emb_final,
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [42]:
final_model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), callbacks=[early])

Epoch 1/15
114/114 [==============================] - 15s 88ms/step - loss: 0.3292 - accuracy: 0.8500 - val_loss: 0.1268 - val_accuracy: 0.9517
Epoch 2/15
114/114 [==============================] - 9s 80ms/step - loss: 0.1133 - accuracy: 0.9586 - val_loss: 0.0983 - val_accuracy: 0.9613
Epoch 3/15
114/114 [==============================] - 8s 67ms/step - loss: 0.0857 - accuracy: 0.9677 - val_loss: 0.0883 - val_accuracy: 0.9659
Epoch 4/15
114/114 [==============================] - 9s 80ms/step - loss: 0.0736 - accuracy: 0.9735 - val_loss: 0.0858 - val_accuracy: 0.9652
Epoch 5/15
114/114 [==============================] - 9s 81ms/step - loss: 0.0570 - accuracy: 0.9812 - val_loss: 0.0823 - val_accuracy: 0.9704
Epoch 6/15
114/114 [==============================] - 8s 70ms/step - loss: 0.0464 - accuracy: 0.9865 - val_loss: 0.0805 - val_accuracy: 0.9697
Epoch 7/15
114/114 [==============================] - 9s 83ms/step - loss: 0.0412 - accuracy: 0.9862 - val_loss: 0.0866 - val_accuracy: 0.969

In [43]:
pred4 = final_model.predict(X_test)

49/49 [==============================] - 6s 97ms/step


In [44]:
new_pred4 = np.round(pred4).astype(int)

In [45]:
print(classification_report(y_test, new_pred4))
print(confusion_matrix(y_test, new_pred4))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1101
           1       0.97      0.93      0.95       451

    accuracy                           0.97      1552
   macro avg       0.97      0.96      0.96      1552
weighted avg       0.97      0.97      0.97      1552

[[1086   15]
 [  33  418]]


In [57]:
def classify_email(user_input):
    preprocessed_input = tf.constant([user_input])

    pred = final_model.predict(preprocessed_input)
    predicted_label = np.round(pred).astype(int)[0][0]

    if predicted_label == 1:
        classification = "SPAM"
    else:
        classification = "NOT SPAM"

    print("The email is classified as:", classification)

user_input = input("Enter the email message to classify: ")
classify_email(user_input)

Enter the email message to classify: dev
1/1 [==============================] - 0s 42ms/step
The email is classified as: NOT SPAM


In [46]:
# Choose best model as you want but for me the last one i.e Pretrained Feature Extractor is best